数据来源于[京东JData算法大赛-高潜用户购买意向预测](https://www.datafountain.cn/competitions/247/datasets).通过Python随机取了10000个用户和500个商品作为项目数据。

In [21]:
import pandas as pd
import sqlite3
from scipy.stats import norm
import random
import numpy as np

In [6]:
data_dir = r'/mnt/data-ubuntu/Projects/user_profile/input'
seed = 123

In [7]:
df_users = pd.read_csv(data_dir + r'/JData/JData_User.csv',encoding='GBK')
df_comments = pd.read_csv(data_dir + r'/JData/JData_Comment.csv',encoding='GBK')
df_products = pd.read_csv(data_dir + r'/JData/JData_Product.csv',encoding='GBK')

In [4]:
# df_users = df_users.sample(10000, random_state = seed).reset_index(drop = True)
# df_products = df_products.sample(200, random_state = seed).reset_index(drop = True)
# df_comments = df_comments.merge(df_products.loc[:, ['sku_id']], on = 'sku_id', how = 'inner')

In [8]:
df_action_02 = pd.read_csv(data_dir + r'/JData/JData_Action_201602.csv',encoding='GBK')
df_action_03 = pd.read_csv(data_dir + r'/JData/JData_Action_201603.csv',encoding='GBK')
df_action_04 = pd.read_csv(data_dir + r'/JData/JData_Action_201604.csv',encoding='GBK')

In [9]:
df_action = pd.concat([df_action_02, df_action_03, df_action_04])

In [12]:
df_action.sku_id.unique().size

28710

In [13]:
df_products.sku_id.unique().size

24187

In [14]:
df_comments.sku_id.unique().size

46546

In [15]:
df_users.user_id.unique().size

105321

In [16]:
df_action.user_id.unique().size

105180

In [56]:
df_sample_p = np.intersect1d(df_products.sku_id.unique(),df_comments.sku_id.unique())
df_sample_p = pd.DataFrame({'sku_id':np.intersect1d(df_action.sku_id.unique(),df_sample_p)}).sample(500,random_state = seed)

In [60]:
df_sample_u = df_action.merge(df_sample_p, on='sku_id', how='inner').user_id.unique()
df_sample_u = pd.DataFrame({'sku_id':np.intersect1d(df_action.user_id.unique(),df_sample_u)})

In [62]:
df_sample_u

,sku_id
0,200001.0
1,200002.0
2,200003.0
3,200004.0
4,200005.0
...,...
87008,305316.0
87009,305317.0
87010,305318.0
87011,305319.0


In [6]:
df_action_02 = df_action_02.merge(df_products.loc[:, ['sku_id']], on = 'sku_id', how = 'inner')
df_action_02 = df_action_02.merge(df_users.loc[:, ['user_id']], on = 'user_id', how = 'inner')

df_action_03 = df_action_03.merge(df_products.loc[:, ['sku_id']], on = 'sku_id', how = 'inner')
df_action_03 = df_action_03.merge(df_users.loc[:, ['user_id']], on = 'user_id', how = 'inner')

df_action_04 = df_action_04.merge(df_products.loc[:, ['sku_id']], on = 'sku_id', how = 'inner')
df_action_04 = df_action_04.merge(df_users.loc[:, ['user_id']], on = 'user_id', how = 'inner')

In [34]:
df_action = pd.concat([df_action_02, df_action_03, df_action_04])
df_users['user_id'] = df_users['user_id'].astype('str')
df_users['sex'] = df_users['sex'].astype('int')
df_comments['sku_id'] = df_comments['sku_id'].astype('str')
df_products['sku_id'] = df_products['sku_id'].astype('str')
df_action['user_id'] = df_action['user_id'].astype('int').astype('str')

In [35]:
# df_users.to_csv(data_dir + r'/user_behavior/users', index = False)
# df_comments.to_csv(data_dir + r'/user_behavior/comments', index = False)
# df_products.to_csv(data_dir + r'/user_behavior/products', index = False)
# df_action.to_csv(data_dir + r'/user_behavior/action', index = False)

In [200]:
conn = sqlite3.connect(data_dir + r'/user_behavior/dw.db')

In [201]:
sql = """
SELECT *
FROM 'dw.actions'
"""
df = pd.read_sql(sql, conn)
df

,user_id,sku_id,time,model_id,type
0,266079,138778,2016-01-31 23:59:02,NaN,1
1,266079,138778,2016-01-31 23:59:03,0.0,6
2,266079,138778,2016-01-31 23:59:40,0.0,6
3,276956,138778,2016-02-01 01:44:12,217.0,6
4,276956,138778,2016-02-01 01:44:12,223.0,6
...,...,...,...,...,...
13015598,253799,156490,2016-04-15 23:46:06,NaN,1
13015599,253799,156490,2016-04-15 23:46:06,26.0,6
13015600,253799,156490,2016-04-15 23:46:37,26.0,6
13015601,253799,156490,2016-04-15 23:48:42,26.0,6


In [202]:
df.sku_id.unique().size

2825

In [204]:
df_orders = df[(df.type == 4)].reset_index(drop=True)

In [207]:
df_orders['time'] = pd.to_datetime(df_orders.loc[:,'time'])

In [208]:
df_orders['hour'] = df_orders.time.dt.hour
df_orders['date'] = df_orders.time.dt.date

In [182]:
df_orders.sort_values(['date', 'hour', 'user_id'], inplace = True)

In [218]:
df_orders[df_orders.user_id == '267362']

,user_id,sku_id,time,model_id,type,hour,date
2704,267362,79520,2016-04-12 12:08:00,NaN,4,12,2016-04-12
4128,267362,81708,2016-04-12 12:07:00,NaN,4,12,2016-04-12
4909,267362,57161,2016-04-12 12:08:00,NaN,4,12,2016-04-12
5188,267362,166707,2016-04-12 12:07:00,NaN,4,12,2016-04-12
5494,267362,88295,2016-04-12 12:07:00,NaN,4,12,2016-04-12


In [225]:
df_temp = df_orders.groupby(['user_id', 'date']).count()['sku_id'][df_orders.groupby(['user_id', 'date']).count()['sku_id']>1]

In [234]:
df_temp.reset_index().groupby('user_id').count()[df_temp.reset_index().groupby('user_id').count()['sku_id']>1]

,date,sku_id
user_id,,


In [239]:
df_temp.reset_index().user_id.unique().size

271

In [224]:
df_orders[df_orders.user_id == '200077']

,user_id,sku_id,time,model_id,type,hour,date
642,200077,147796,2016-03-15 12:27:00,NaN,4,12,2016-03-15
5460,200077,88295,2016-03-15 11:48:00,NaN,4,11,2016-03-15
5461,200077,88295,2016-03-15 11:49:00,NaN,4,11,2016-03-15
5462,200077,88295,2016-03-15 11:53:00,NaN,4,11,2016-03-15


In [133]:
sql = """
SELECT *
FROM 'dw.products'
"""
df_p = pd.read_sql(sql, conn)
df_p.dtypes

sku_id     object
a1          int64
a2          int64
a3          int64
cate        int64
brand       int64
price     float64
dtype: object

In [98]:
sql = """
SELECT *
FROM 'dw.products'
"""
df = pd.read_sql(sql, conn)
df

,sku_id,a1,a2,a3,cate,brand,price
0,101134,2,1,2,8,124,82.94
1,10130,2,1,2,8,812,16.33
2,101443,3,1,2,8,214,18.72
3,10224,3,2,2,8,214,36.09
4,102527,3,1,2,8,214,38.46
...,...,...,...,...,...,...,...
495,98714,1,2,1,8,885,23.89
496,98963,3,-1,-1,8,214,72.18
497,99639,3,1,1,8,214,27.05
498,99774,1,1,2,8,403,91.56


In [73]:
[random.randint(500, 10000)/100 for i in range(500)]

[5.26,
 25.64,
 78.39,
 45.15,
 34.59,
 32.82,
 98.78,
 21.19,
 86.7,
 71.51,
 9.22,
 26.16,
 18.51,
 35.26,
 8.67,
 69.0,
 29.14,
 75.32,
 60.97,
 43.33,
 78.17,
 81.49,
 90.88,
 88.26,
 25.93,
 99.03,
 90.17,
 34.67,
 54.56,
 35.43,
 57.54,
 32.31,
 10.87,
 70.69,
 81.59,
 5.64,
 21.74,
 90.33,
 59.83,
 94.92,
 23.11,
 6.92,
 48.32,
 64.83,
 37.71,
 7.38,
 61.29,
 19.69,
 83.43,
 85.03,
 90.4,
 13.25,
 31.55,
 56.46,
 71.66,
 71.46,
 62.97,
 88.64,
 52.55,
 73.48,
 99.17,
 18.61,
 46.11,
 12.09,
 80.68,
 53.17,
 21.45,
 13.16,
 89.43,
 80.87,
 85.17,
 26.05,
 30.93,
 79.16,
 27.43,
 45.76,
 45.18,
 64.57,
 79.64,
 16.56,
 36.9,
 75.05,
 95.69,
 55.06,
 89.43,
 15.21,
 10.41,
 87.75,
 76.21,
 77.19,
 47.73,
 75.0,
 77.33,
 96.64,
 36.43,
 33.66,
 69.18,
 23.33,
 97.8,
 34.76,
 71.54,
 9.51,
 30.07,
 47.43,
 73.22,
 42.9,
 73.26,
 84.01,
 16.43,
 9.28,
 62.02,
 11.82,
 29.48,
 87.58,
 7.6,
 71.75,
 98.58,
 14.32,
 13.48,
 99.31,
 9.59,
 63.63,
 86.06,
 64.13,
 39.74,
 8.97,
 38.66,
 10